# Create individual calculators for each lab group using BL data

In [1]:
# Set up
import pandas as pd
import numpy as np
import sys
from pathlib import Path
CODE_ROOT = Path.cwd().parents[1]
sys.path.append(str(CODE_ROOT))
import config
from openpyxl import load_workbook
from openpyxl.formatting.rule import FormulaRule
from openpyxl.styles import Font, PatternFill
from openpyxl.utils import range_boundaries
import os
import shutil
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [2]:
# Load data
labs = pd.read_csv(config.BL_RAW_SAMPLE / "final_sample_with_BL_file_status.csv")

In [3]:
# Calculator template path
calculator_template = config.DATA_ROOT / "13_Calculators" / "1_Templates" / "calculator_v1.xlsx"

# BL surveys folder
bl_surveys_folder = config.BL_RAW_SURVEY / "1_LabExcels"

# Calculator output folder
calculator_output_folder = config.DATA_ROOT / "13_Calculators" / "2_Individualized"

In [4]:
# List of labs to create calculators for
labs_create_calc = labs[labs["file_filled"] == True]

# Track missing surveys
missing_surveys = []

# For each labgroupid, create a calculator and save in individualized folder
for labgroupid, group in labs_create_calc.groupby("labgroupid"):

    # Survey file path
    bl_survey = os.path.join(
        bl_surveys_folder,
        f"BL_{labgroupid}.xlsx"
    )

    if not os.path.exists(bl_survey):
        missing_surveys.append(labgroupid)
        print(f"Missing survey for {labgroupid}")
        continue

    # Create a subfolder for this labgroup inside the output folder
    lab_output_folder = os.path.join(calculator_output_folder, str(labgroupid))
    os.makedirs(lab_output_folder, exist_ok=True)

    # Output calculator file path
    output_file = os.path.join(
        lab_output_folder,
        f"Energy_Use_Report.xlsx"
    )

    # Copy the template to create a new calculator file
    shutil.copy(calculator_template, output_file)

    # Load BL survey workbook and calculator workbook
    bl_wb = load_workbook(bl_survey, data_only=True)
    calc_wb = load_workbook(output_file)

    # Define sheets and ranges to transfer
    sheet_mappings = {
        "3. Fridges (4°C)": {"dest_sheet": "Fridges (4°C)", "range": "C6:L9"},
        "4. Freezers (-20°C)": {"dest_sheet": "Freezers (-20°C)", "range": "C6:V13"},
        "5. ULT Freezers": {"dest_sheet": "ULT Freezers", "range": "C6:L14"},
        "6. Glassware Drying Cabinets": {"dest_sheet": "Glassware Drying Cabinets", "range": "C6:L15"},
        "7. Microbio Safety Cabinets": {"dest_sheet": "Microbio Safety Cabinets", "range": "C6:L14"},
        "8. Cryostats": {"dest_sheet": "Cryostats", "range": "C6:L13"},
        "9. Water Baths": {"dest_sheet": "Water Baths", "range": "C6:L16"},
        "10. CO2 Incubators": {"dest_sheet": "CO2 Incubators", "range": "C6:L11"},
        "11. Block Heaters": {"dest_sheet": "Block Heaters", "range": "C6:L14"},
        "12. IT Equipment": {"dest_sheet": "IT Equipment", "range": "C6:L14"},
    }

    # Transfer values
    for src_sheet, info in sheet_mappings.items():
        bl_ws = bl_wb[src_sheet]
        calc_ws = calc_wb[info["dest_sheet"]]
        min_col, min_row, max_col, max_row = range_boundaries(info["range"])

        for row in range(min_row, max_row + 1):
            for col in range(min_col, max_col + 1):
                calc_ws.cell(row=row, column=col).value = bl_ws.cell(row=row, column=col).value


    # Save the updated calculator
    calc_wb.save(output_file)

# Report missing surveys
# After loop
if missing_surveys:
    print("The following labs were skipped due to missing surveys:")
    for lab in missing_surveys:
        print(f" - {lab}")